# Times

## Introduction

The concept of *time* is fundamental to any kind of time series analysis and simulation. This is why we shall dedicate quite a lot of thought to its representation in thalesians.tsa.

## `xtimes`

`xtimes` is a generator (and, by implication, an iterator) designed to be a source of times to thalesians.tsa routines.